In [27]:
import pandas as pd

Подведите результаты эксперимента в экселе по следующим данным: ab_stats.csv - Google Диск: Стат значимо ли отличается ARPPU в двух группах ? Какие рекомендации дадите менеджеру?

In [28]:
from scipy.stats import ttest_1samp, norm, t, kstest, shapiro, mannwhitneyu #импорт библиотек по статистике
import statsmodels.api as sm #импорт библиотек по статистике
import numpy as np # библиотека для мат.функций.

import warnings # код убирает предупреждения
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

import matplotlib.pyplot as plt # для графиков
import seaborn as sns # для графиков
import pandas as pd # для работы сданными

In [29]:
data = pd.read_csv('C:/Users/Евгений/Desktop/Итоговая работа часть 2/ab_stats (1).csv', sep=',')
data.head(10)

,revenue,num_purchases,purchase,ab_group,av_site visit
0,0.0,0,0,A,9.040174
1,0.0,0,0,A,4.811628
2,0.0,0,0,A,7.342623
3,0.0,0,0,A,7.744581
4,0.0,0,0,A,10.511814
5,0.0,0,0,A,9.578727
6,0.0,0,0,A,6.162601
7,0.0,0,0,A,11.909452
8,0.0,0,0,A,6.540910
9,0.0,0,0,A,7.990794


In [30]:
data


,revenue,num_purchases,purchase,ab_group,av_site visit
0,0.0,0,0,A,9.040174
1,0.0,0,0,A,4.811628
2,0.0,0,0,A,7.342623
3,0.0,0,0,A,7.744581
4,0.0,0,0,A,10.511814
...,...,...,...,...,...
23647,0.0,0,0,B,10.207152
23648,0.0,0,0,B,5.148761
23649,0.0,0,0,B,4.809019
23650,0.0,0,0,B,8.127488


In [31]:
data = data.rename(columns = {'av_site visit':'av_site_visit'})

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23652 entries, 0 to 23651
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   revenue        23652 non-null  float64
 1   num_purchases  23652 non-null  int64  
 2   purchase       23652 non-null  int64  
 3   ab_group       23652 non-null  object 
 4   av_site_visit  23652 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 924.0+ KB


In [33]:
data.describe()

,revenue,num_purchases,purchase,av_site_visit
count,23652.000000,23652.000000,23652.000000,23652.000000
mean,0.324689,0.043590,0.020717,7.013112
std,9.557730,1.079403,0.142438,3.154584
min,0.000000,0.000000,0.000000,-12.073486
25%,0.000000,0.000000,0.000000,5.173787
50%,0.000000,0.000000,0.000000,7.007936
75%,0.000000,0.000000,0.000000,8.864119
max,1303.609284,152.000000,1.000000,22.446822


In [34]:
data.query('ab_group == "B" and av_site_visit < 0')

,revenue,num_purchases,purchase,ab_group,av_site_visit
11866,0.0,0,0,B,-1.403294
11960,0.0,0,0,B,-1.312672
11970,0.0,0,0,B,-0.980742
12018,0.0,0,0,B,-0.048983
12067,0.0,0,0,B,-0.097441
...,...,...,...,...,...
23567,0.0,0,0,B,-3.901748
23606,0.0,0,0,B,-2.284448
23626,0.0,0,0,B,-3.528928
23634,0.0,0,0,B,-1.708265


In [35]:
data.query('ab_group == "A" and av_site_visit < 0')

,revenue,num_purchases,purchase,ab_group,av_site_visit
1789,0.0,0,0,A,-12.073486
1856,0.0,0,0,A,-0.683411
5122,0.0,0,0,A,-0.045455
7109,0.0,0,0,A,-12.073486
8118,0.0,0,0,A,-0.126946
9770,0.0,0,0,A,-0.045455


In [36]:
(462+6)/23652

0.0197869101978691

In [37]:
data_clean = data.query('av_site_visit >= 0')

In [38]:
data_clean.describe()

,revenue,num_purchases,purchase,av_site_visit
count,23184.000000,23184.000000,23184.000000,23184.000000
mean,0.326511,0.043780,0.020704,7.188254
std,9.645839,1.089462,0.142394,2.924011
min,0.000000,0.000000,0.000000,0.008679
25%,0.000000,0.000000,0.000000,5.314379
50%,0.000000,0.000000,0.000000,7.068656
75%,0.000000,0.000000,0.000000,8.910900
max,1303.609284,152.000000,1.000000,22.446822


In [39]:
data_clean.ab_group.value_counts()

ab_group
A    11829
B    11355
Name: count, dtype: int64

In [40]:
data_clean.query('ab_group=="A"').describe()

,revenue,num_purchases,purchase,av_site_visit
count,11829.000000,11829.000000,11829.000000,11829.000000
mean,0.404667,0.050723,0.021642,6.980379
std,13.136545,1.467883,0.145517,2.004450
min,0.000000,0.000000,0.000000,0.145466
25%,0.000000,0.000000,0.000000,5.657590
50%,0.000000,0.000000,0.000000,6.984166
75%,0.000000,0.000000,0.000000,8.346243
max,1303.609284,152.000000,1.000000,17.728836


In [41]:
data_clean.query('ab_group=="B"').describe()

,revenue,num_purchases,purchase,av_site_visit
count,11355.000000,11355.000000,11355.000000,11355.000000
mean,0.245093,0.036548,0.019727,7.404806
std,3.192299,0.422820,0.139067,3.630412
min,0.000000,0.000000,0.000000,0.008679
25%,0.000000,0.000000,0.000000,4.746728
50%,0.000000,0.000000,0.000000,7.262590
75%,0.000000,0.000000,0.000000,9.889165
max,113.830000,25.000000,1.000000,22.446822


По предварительному просмотру данных можно сделать вывод, что группа А себя показывает лучше как в прибыли, так и в кол-ве покупок

In [42]:
data_clean.purchase.value_counts()

purchase
0    22704
1      480
Name: count, dtype: int64

In [43]:
data_clean.query('ab_group=="A"').purchase.value_counts()

purchase
0    11573
1      256
Name: count, dtype: int64

In [44]:
data_clean.query('ab_group=="B"').purchase.value_counts()

purchase
0    11131
1      224
Name: count, dtype: int64

In [45]:
import plotly.express as px
fig = px.histogram(data_clean, x='revenue', color = 'ab_group',title='revenue_distribution', marginal = 'box' , nbins = 100)

fig.show(renderer="colab")

In [46]:
mw_stats = mannwhitneyu(x=data_clean.query('ab_group=="A"').revenue.values,
                                                  y=data_clean.query('ab_group=="B"').revenue.values)
mw_stats

MannwhitneyuResult(statistic=67287553.0, pvalue=0.306808963067974)

p value больше чем 0.05 значит статистически значимых различий между группами нет, но выбираем группу А, так как она выглядит чуть лучше